# Mineração de Dados (Clusterização K-MEANS)

### 1. Carregar o dataset tratado

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore', category=FutureWarning) 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. CARREGAR E AGREGAR O DATASET
try:
    dados = pd.read_excel("dataset_tratado.xlsx") 
except FileNotFoundError:
    print("ERRO: Arquivo 'dataset_tratado.xlsx' não encontrado.")
    exit()

# Agregação: Transforma transações (várias linhas) em perfis de produto (uma linha por produto)
df_cluster = dados.groupby('Produto').agg({
    'Preço Unitário': 'mean', # Preço Médio do Produto
    'Quantidade Vendida': 'sum' # Volume Total vendido
}).reset_index() 

print("✅ Dados agregados. df_cluster pronto para ML.")

In [ ]:
# 3. NORMALIZAÇÃO: Padronizar os dados
colunas_para_cluster = ['Preço Unitário', 'Quantidade Vendida']
scaler = StandardScaler()

# O fit_transform calcula a nova escala e aplica nos dados
X_scaled = scaler.fit_transform(df_cluster[colunas_para_cluster])

print("✅ Dados normalizados.")

In [ ]:
# 4. APLICAÇÃO DO K-MEANS
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10) 
df_cluster['Cluster'] = kmeans.fit_predict(X_scaled)

print("✅ K-Means concluído. Gerando visualização...")

# 5. VISUALIZAÇÃO DOS GRUPOS
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_cluster, 
                x='Preço Unitário', 
                y='Quantidade Vendida', 
                hue='Cluster', 
                palette='viridis', 
                s=100)
plt.title("Clusterização: Segmentação de Produtos (Preço vs Volume)", fontsize=14)
plt.xlabel("Preço Médio do Produto (R$)")
plt.ylabel("Quantidade Total Vendida")
plt.grid(True, linestyle='--', alpha=0.3)
plt.legend(title='Grupo')
plt.show()

In [ ]:
# 6. INTERPRETAÇÃO DOS CENTROS (Conversão de volta para R$ e Qtd)
centros = scaler.inverse_transform(kmeans.cluster_centers_)
centros_df = pd.DataFrame(centros, columns=['Preço Médio', 'Qtd Total'])

print("\n--- CENTROS DOS CLUSTERS (PERFIS MÉDIOS) ---")
print("Estes valores definem o perfil de cada grupo:")
print(centros_df.sort_values(by='Preço Médio', ascending=False).round(0))